In [19]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time
import mysql.connector  # mysql.connector 모듈을 임포트합니다.
from datetime import datetime  # 날짜 형식 변환을 위한 모듈

# 웹드라이버 초기화
driver = webdriver.Chrome()

# URL로 이동
url = "https://www.kebhana.com/cms/rate/index.do?contentUrl=/cms/rate/wpfxd651_09i.do#//HanaBank"
driver.get(url)

# 페이지 로딩 대기
time.sleep(10)  # 페이지 로딩 시간을 위한 대기. 필요에 따라 조정

# 통화 선택 드롭다운 메뉴 선택 (value 기반으로 선택, 예: 'EUR' 선택)
currency_dropdown = Select(driver.find_element(By.CSS_SELECTOR, '#curCd'))
currency_dropdown.select_by_value('CNY')  # 'EUR' 값에 해당하는 옵션 선택

# 조회 버튼 클릭
search_button = driver.find_element(By.CSS_SELECTOR, '#HANA_CONTENTS_DIV > div.btnBoxCenter > a')
search_button.click()

# 선택 후 데이터가 갱신될 시간을 기다림
time.sleep(10)

# 날짜 추출 (nth-child(2)부터 nth-child(335)까지)
date_row = driver.find_element(By.CSS_SELECTOR, '#numLookHis2 > table > tbody > tr:nth-child(1)')
dates = date_row.find_elements(By.CSS_SELECTOR, 'th')

# Open, High, Low, Close 위치 추출 (nth-child(2)부터 nth-child(335)까지)
open_row = driver.find_elements(By.CSS_SELECTOR, '#chart01Tr0 > td')
high_row = driver.find_elements(By.CSS_SELECTOR, '#chart01Tr1 > td')
low_row = driver.find_elements(By.CSS_SELECTOR, '#chart01Tr2 > td')
close_row = driver.find_elements(By.CSS_SELECTOR, '#chart01Tr3 > td')

# 데이터를 저장할 리스트 초기화
data_list = []

# 날짜와 가격 데이터를 추출
for i in range(1, len(dates)):  # td:nth-child(2)부터 시작
    try:
        # 날짜 추출
        date_text = dates[i].text.strip()

        # 날짜 형식을 "YYYY-MM-DD"로 변환 (예시: 20230524 -> 2023-05-24)
        if len(date_text) == 8:  # 날짜 형식이 8자리일 경우
            date_formatted = datetime.strptime(date_text, '%Y%m%d').strftime('%Y-%m-%d')
        else:
            date_formatted = date_text  # 형식에 맞지 않으면 그대로 사용

        # 각 가격 데이터 추출
        open_price = open_row[i-1].text.strip().replace(',', '')
        high_price = high_row[i-1].text.strip().replace(',', '')
        low_price = low_row[i-1].text.strip().replace(',', '')
        close_price = close_row[i-1].text.strip().replace(',', '')

        # 데이터 저장
        data_list.append({
            'date': date_formatted,  # 변환된 날짜를 사용
            'open': open_price if open_price else 'N/A',
            'high': high_price if high_price else 'N/A',
            'low': low_price if low_price else 'N/A',
            'close': close_price if close_price else 'N/A',
            'state': 'CNY'
        })

    except Exception as e:
        print(f"Error processing data for index {i}: {e}")

# 데이터 출력 확인
for data in data_list:
    print(data)

# MySQL 연결 정보 설정 (MySQL에 데이터 삽입 부분 활성화 시 사용)
config = {
    'user': 'yangjinan',
    'password': 'ALsk85200!!',
    'host': 'hanaex.mysql.database.azure.com',
    'database': 'my_database',
    'port': 3306
}

# MySQL 연결
conn = mysql.connector.connect(**config)
cursor = conn.cursor()

# 데이터 삽입
for data in data_list:
    insert_query = """
    INSERT INTO hr_ex_table (date, open, high, low, close, state)
    VALUES (%s, %s, %s, %s, %s, %s)
    """
    cursor.execute(insert_query, (
        data['date'],
        float(data['open']) if data['open'] != 'N/A' else None,  # 'N/A' 값 처리
        float(data['high']) if data['high'] != 'N/A' else None,
        float(data['low']) if data['low'] != 'N/A' else None,
        float(data['close']) if data['close'] != 'N/A' else None,
        data['state']
    ))
    print(data)

# 변경사항 커밋
conn.commit()

# 연결 종료
cursor.close()
conn.close()

# 웹드라이버 종료
driver.quit()

{'date': '2023-05-12', 'open': '190.44', 'high': '192.31', 'low': '190.44', 'close': '192.02', 'state': 'CNY'}
{'date': '2023-05-15', 'open': '191.86', 'high': '192.48', 'low': '191.54', 'close': '192.48', 'state': 'CNY'}
{'date': '2023-05-16', 'open': '192.01', 'high': '192.09', 'low': '191.57', 'close': '191.71', 'state': 'CNY'}
{'date': '2023-05-17', 'open': '191.72', 'high': '191.86', 'low': '190.67', 'close': '190.93', 'state': 'CNY'}
{'date': '2023-05-18', 'open': '191.29', 'high': '191.29', 'low': '189.47', 'close': '189.71', 'state': 'CNY'}
{'date': '2023-05-19', 'open': '189.72', 'high': '190.19', 'low': '188.12', 'close': '189.08', 'state': 'CNY'}
{'date': '2023-05-22', 'open': '188.66', 'high': '188.85', 'low': '186.99', 'close': '187.02', 'state': 'CNY'}
{'date': '2023-05-23', 'open': '187.69', 'high': '187.69', 'low': '185.57', 'close': '186.98', 'state': 'CNY'}
{'date': '2023-05-24', 'open': '185.76', 'high': '187.21', 'low': '185.76', 'close': '186.98', 'state': 'CNY'}
{

{'date': '2023-05-24', 'open': '185.76', 'high': '187.21', 'low': '185.76', 'close': '186.98', 'state': 'CNY'}
{'date': '2023-05-25', 'open': '186.32', 'high': '187.68', 'low': '186.32', 'close': '187.62', 'state': 'CNY'}
{'date': '2023-05-26', 'open': '186.93', 'high': '187.96', 'low': '186.93', 'close': '187.51', 'state': 'CNY'}
{'date': '2023-05-30', 'open': '187.25', 'high': '187.25', 'low': '186.06', 'close': '186.45', 'state': 'CNY'}
{'date': '2023-05-31', 'open': '186.31', 'high': '186.55', 'low': '185.64', 'close': '186.30', 'state': 'CNY'}
{'date': '2023-06-01', 'open': '185.92', 'high': '185.92', 'low': '185.31', 'close': '185.51', 'state': 'CNY'}
{'date': '2023-06-02', 'open': '185.58', 'high': '185.58', 'low': '184.06', 'close': '184.26', 'state': 'CNY'}
{'date': '2023-06-05', 'open': '184.32', 'high': '184.32', 'low': '183.20', 'close': '183.35', 'state': 'CNY'}
{'date': '2023-06-07', 'open': '183.99', 'high': '183.99', 'low': '182.16', 'close': '183.16', 'state': 'CNY'}
{

{'date': '2023-09-08', 'open': '182.02', 'high': '182.04', 'low': '181.15', 'close': '181.54', 'state': 'CNY'}
{'date': '2023-09-11', 'open': '181.35', 'high': '182.40', 'low': '181.21', 'close': '181.93', 'state': 'CNY'}
{'date': '2023-09-12', 'open': '181.82', 'high': '181.95', 'low': '181.30', 'close': '181.95', 'state': 'CNY'}
{'date': '2023-09-13', 'open': '181.46', 'high': '182.76', 'low': '181.46', 'close': '182.59', 'state': 'CNY'}
{'date': '2023-09-14', 'open': '181.90', 'high': '182.82', 'low': '181.90', 'close': '182.15', 'state': 'CNY'}
{'date': '2023-09-15', 'open': '182.26', 'high': '182.89', 'low': '182.17', 'close': '182.83', 'state': 'CNY'}
{'date': '2023-09-18', 'open': '182.58', 'high': '182.72', 'low': '181.64', 'close': '181.88', 'state': 'CNY'}
{'date': '2023-09-19', 'open': '182.06', 'high': '182.13', 'low': '181.01', 'close': '182.13', 'state': 'CNY'}
{'date': '2023-09-20', 'open': '181.28', 'high': '182.23', 'low': '181.28', 'close': '182.01', 'state': 'CNY'}
{

{'date': '2024-01-09', 'open': '183.47', 'high': '184.07', 'low': '182.73', 'close': '183.86', 'state': 'CNY'}
{'date': '2024-01-10', 'open': '182.96', 'high': '184.01', 'low': '182.96', 'close': '183.82', 'state': 'CNY'}
{'date': '2024-01-11', 'open': '183.83', 'high': '183.83', 'low': '183.18', 'close': '183.61', 'state': 'CNY'}
{'date': '2024-01-12', 'open': '183.51', 'high': '183.57', 'low': '182.56', 'close': '182.56', 'state': 'CNY'}
{'date': '2024-01-15', 'open': '183.35', 'high': '184.10', 'low': '182.73', 'close': '184.07', 'state': 'CNY'}
{'date': '2024-01-16', 'open': '183.63', 'high': '185.32', 'low': '183.63', 'close': '185.32', 'state': 'CNY'}
{'date': '2024-01-17', 'open': '184.70', 'high': '186.63', 'low': '184.44', 'close': '186.36', 'state': 'CNY'}
{'date': '2024-01-18', 'open': '186.19', 'high': '186.43', 'low': '185.29', 'close': '185.76', 'state': 'CNY'}
{'date': '2024-01-19', 'open': '186.06', 'high': '186.06', 'low': '184.91', 'close': '185.56', 'state': 'CNY'}
{

{'date': '2024-05-03', 'open': '190.31', 'high': '190.31', 'low': '188.60', 'close': '189.05', 'state': 'CNY'}
{'date': '2024-05-07', 'open': '189.55', 'high': '189.55', 'low': '187.73', 'close': '188.03', 'state': 'CNY'}
{'date': '2024-05-08', 'open': '188.13', 'high': '189.04', 'low': '188.13', 'close': '188.82', 'state': 'CNY'}
{'date': '2024-05-09', 'open': '188.74', 'high': '189.71', 'low': '188.74', 'close': '189.14', 'state': 'CNY'}
{'date': '2024-05-10', 'open': '189.20', 'high': '189.44', 'low': '189.00', 'close': '189.38', 'state': 'CNY'}
{'date': '2024-05-13', 'open': '189.27', 'high': '189.76', 'low': '188.86', 'close': '188.86', 'state': 'CNY'}
{'date': '2024-05-14', 'open': '189.21', 'high': '189.24', 'low': '188.83', 'close': '188.91', 'state': 'CNY'}
{'date': '2024-05-16', 'open': '189.08', 'high': '189.08', 'low': '186.17', 'close': '186.72', 'state': 'CNY'}
{'date': '2024-05-17', 'open': '186.89', 'high': '187.99', 'low': '186.60', 'close': '187.44', 'state': 'CNY'}
{

{'date': '2024-08-30', 'open': '187.56', 'high': '188.72', 'low': '187.56', 'close': '188.51', 'state': 'CNY'}
{'date': '2024-09-02', 'open': '188.35', 'high': '188.80', 'low': '188.16', 'close': '188.20', 'state': 'CNY'}
{'date': '2024-09-03', 'open': '188.54', 'high': '188.87', 'low': '187.99', 'close': '188.23', 'state': 'CNY'}
{'date': '2024-09-04', 'open': '188.31', 'high': '188.94', 'low': '187.68', 'close': '187.76', 'state': 'CNY'}
{'date': '2024-09-05', 'open': '188.78', 'high': '188.78', 'low': '187.67', 'close': '188.39', 'state': 'CNY'}
{'date': '2024-09-06', 'open': '188.05', 'high': '188.32', 'low': '187.26', 'close': '188.05', 'state': 'CNY'}
{'date': '2024-09-09', 'open': '187.74', 'high': '189.34', 'low': '187.74', 'close': '188.75', 'state': 'CNY'}
{'date': '2024-09-10', 'open': '188.51', 'high': '188.90', 'low': '188.19', 'close': '188.42', 'state': 'CNY'}
{'date': '2024-09-11', 'open': '188.69', 'high': '188.69', 'low': '187.94', 'close': '188.39', 'state': 'CNY'}
{